In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("E:\Coding\my_stuff"))
from my_keys import MufexKeys # type: ignore
import numpy as np
from datetime import datetime

from strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.backtest import run_df_backtest, or_backtest

from quantfreedom.exchanges.mufex_exchange.mufex import Mufex

mufex_main = Mufex(
    api_key=MufexKeys.mainnet_neo_api_key,
    secret_key=MufexKeys.mainnet_neo_secret_key,
    use_test_net=False
)
symbol = 'BTCUSDT'

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)

In [4]:
dos_arrays = DynamicOrderSettings(
    max_trades=np.array([4]),
    account_pct_risk_per_trade=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

[autoreload of quantfreedom.strategies.strategy failed: Traceback (most recent call last):
  File "e:\Coding\venvs\qfree\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "e:\Coding\venvs\qfree\lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "E:\Coding\backtesters\QuantFreedom\quantfreedom\strategies\strategy.py", line 9, in <module>
    class Strategy:
  File "E:\Coding\backtesters\QuantFreedom\quantfreedom\strategies\strategy.py", line 28, in Strategy
    dos_arrays: DynamicOrderSettings,
NameErro

In [3]:
long_strat = RSIRisingFalling(
    long_short="long",
    dos_arrays=dos_arrays,
    rsi_length=np.array([14]),
    rsi_is_below=np.array([40, 60, 80]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

Exception long_set_entries_exits_array -> 'IndicatorSettings' object is not callable


Exception: Exception long_set_entries_exits_array -> 'IndicatorSettings' object is not callable

In [8]:
backtest_settings = BacktestSettings()

exchange_settings = mufex_main.set_and_get_exchange_settings(
    leverage_mode=LeverageModeType.Isolated,
    position_mode=PositionModeType.HedgeMode,
    symbol=symbol,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)



In [5]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    loop_start=0,
    loop_end=50,
)

In [6]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

In [9]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=0,
    plot_results=False,
    logger_bool=True,
)

DynamicOrderSettings(
    max_trades = 4,
    account_pct_risk_per_trade = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.001,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)
IndicatorSettings(
    rsi_is_above = nan,
    rsi_is_below = 80.0,
    rsi_length = 14,
)


In [8]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,0,50,1710409500000,2024-03-14 09:45:00,EntryFilled,1000.000,932.970,7335.215,67.030,...,7402.245,73356.9,NaN,0.101,7402.245,NaN,0.3,73147.5,0.9,74017.7
1,2,0,52,1710410100000,2024-03-14 09:55:00,EntryFilled,1000.000,791.470,23131.398,208.530,...,8602.468,73315.3,NaN,0.218,16004.713,NaN,0.3,73147.5,0.8,73958.5
2,2,0,55,1710411000000,2024-03-14 10:10:00,StopLossFilled,938.351,938.351,NaN,NaN,...,NaN,NaN,73147.5,NaN,NaN,-61.649,NaN,NaN,NaN,NaN
3,2,0,57,1710411600000,2024-03-14 10:20:00,EntryFilled,938.351,884.941,4967.883,53.410,...,5021.293,73237.9,NaN,0.069,5021.293,NaN,0.4,72917.2,1.2,74120.9
4,2,0,59,1710412200000,2024-03-14 10:30:00,EntryFilled,938.351,783.770,14113.320,154.581,...,4278.725,73309.2,NaN,0.127,9300.018,NaN,0.5,72917.2,1.3,74208.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2,0,2942,1711277100000,2024-03-24 10:45:00,TakeProfitFilled,60.321,60.321,NaN,NaN,...,NaN,NaN,65321.0,NaN,NaN,9.497,NaN,NaN,NaN,NaN
584,2,0,2948,1711278900000,2024-03-24 11:15:00,EntryFilled,60.321,58.842,155.761,1.479,...,157.240,65158.2,NaN,0.002,157.240,NaN,0.5,64821.8,1.4,66045.9
585,2,0,2953,1711280400000,2024-03-24 11:40:00,EntryFilled,60.321,55.868,461.530,4.453,...,152.982,65169.8,NaN,0.004,310.222,NaN,0.5,64821.8,1.4,66061.1
586,2,0,2955,1711281000000,2024-03-24 11:50:00,EntryFilled,60.321,51.434,912.807,8.887,...,149.942,65306.4,NaN,0.006,460.164,NaN,0.5,64861.9,1.4,66106.4
